<a href="https://colab.research.google.com/github/MJMortensonWarwick/data_engineering_for_data_scientists/blob/main/1_2_Medallion_Architecture_Data_Lakehouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.2 Medallion Architecture / Data Lakehouse
This Notebook works similarly to our ETL Notebook, but explicity creates three layers to our data lake (bronze, silver, gold).

We will relax some of the previous rigour to make this a more natural Notebook progression (e.g. no `utilities.py` file and so on).

We'll start by creating some raw data and saving it to the bronze layer (which we will also make).


In [1]:
import pandas as pd
import numpy as np
import os
import json
import duckdb

# Create our "Data Lake" directories
os.makedirs("datalake/bronze", exist_ok=True)
os.makedirs("datalake/silver", exist_ok=True)
os.makedirs("datalake/gold", exist_ok=True)

# 1. Generate "Messy" Raw Data
def generate_bronze_data():
    np.random.seed(42)
    data = []

    # We simulate 1000 raw events
    for i in range(1000):
        event = {
            "transaction_id": str(i) if np.random.rand() > 0.05 else None, # 5% missing IDs
            "user_id": np.random.randint(100, 110), # Only 10 users
            "amount": np.random.choice([10.5, 20.0, "50.0", -5.0, None]), # Mixed types & bad data
            "timestamp": "2024-02-10T10:00:00" if i % 2 == 0 else "2024-02-10 10:01:00", # Inconsistent dates
            "raw_metadata": "{\"source\": \"mobile\"}" # Nested JSON string
        }
        # Duplicate some rows to simulate ingestion errors
        if i % 50 == 0:
            data.append(event)
        data.append(event)

    # Save as Raw JSON (Line Delimited) - The "Bronze" format
    # Bronze is often just a "dump" of the source system.
    file_path = "datalake/bronze/raw_transactions.json"
    with open(file_path, "w") as f:
        for entry in data:
            f.write(json.dumps(entry) + "\n")

    print(f"BRONZE LAYER: Generated {len(data)} raw records at {file_path}")

generate_bronze_data()

BRONZE LAYER: Generated 1020 raw records at datalake/bronze/raw_transactions.json


### Bronze to Silver (Cleaning & Validation)
Goal: Clean the data, enforce types, and deduplicate. Save as Parquet.

Why Parquet? It compresses data and stores schema information, making it much faster to read than JSON.

In [2]:
def process_silver():
    print("\nPROCESSING: Bronze -> Silver")

    # 1. Read Raw Data
    df = pd.read_json("datalake/bronze/raw_transactions.json", lines=True)

    # 2. Data Cleaning Rules
    # Drop duplicates
    df = df.drop_duplicates()

    # Fix 'amount': Convert to numeric, coerce errors (strings) to NaN, fill NaNs with 0
    df["amount"] = pd.to_numeric(df["amount"], errors='coerce').fillna(0)

    # Filter out negative amounts (Business Rule)
    df = df[df["amount"] >= 0]

    # Fix Timestamp format
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    # 3. Enforce Schema & Save as Parquet
    # This ensures that downstream users don't have to guess data types.
    output_path = "datalake/silver/clean_transactions.parquet"
    df.to_parquet(output_path, index=False)

    print(f"SILVER LAYER: Saved {len(df)} clean records to {output_path}")
    print(df.head(3))

process_silver()


PROCESSING: Bronze -> Silver
SILVER LAYER: Saved 783 clean records to datalake/silver/clean_transactions.parquet
   transaction_id  user_id  amount           timestamp          raw_metadata
0             0.0      107     0.0 2024-02-10 10:00:00  {"source": "mobile"}
2             1.0      109    50.0 2024-02-10 10:01:00  {"source": "mobile"}
3             2.0      107     0.0 2024-02-10 10:00:00  {"source": "mobile"}


### Silver to Gold (Aggregation & Business Logic)
Goal: Create a business-level aggregate table (e.g., "Daily Sales per User").

This layer is highly curated. It is usually "read-optimised" for dashboards or engineered features for data science models.

In [3]:
def process_gold():
    print("\nPROCESSING: Silver -> Gold")

    # 1. Read Silver Data (Fast Parquet Read)
    df = pd.read_parquet("datalake/silver/clean_transactions.parquet")

    # 2. Aggregation (The "Business Logic")
    gold_df = df.groupby("user_id").agg(
        total_spent=("amount", "sum"),
        transaction_count=("transaction_id", "count"),
        last_seen=("timestamp", "max")
    ).reset_index()

    # 3. Save to Gold
    output_path = "datalake/gold/user_daily_summary.parquet"
    gold_df.to_parquet(output_path, index=False)

    print(f"GOLD LAYER: Saved {len(gold_df)} summary records to {output_path}")
    print(gold_df.head(3))

process_gold()


PROCESSING: Silver -> Gold
GOLD LAYER: Saved 10 summary records to datalake/gold/user_daily_summary.parquet
   user_id  total_spent  transaction_count           last_seen
0      100       1609.0                 71 2024-02-10 10:01:00
1      101       1405.5                 61 2024-02-10 10:01:00
2      102       1688.0                 78 2024-02-10 10:01:00


### Data Lakehouse
Lastly, we will build a data lakehouse using DuckDB. Note, this is a virtual database that does not actually store the data. Instead we infer the schema, and return data, from the parquet files we saved in the Gold layer.

In [4]:
print("\nANALYST QUERY (DuckDB Lakehouse)")

# We use DuckDB to query the file directly.
# This mimics a "Serverless SQL" experience.

query = """
    SELECT
        user_id,
        total_spent,
        CASE
            WHEN total_spent > 1000 THEN 'VIP'
            ELSE 'Regular'
        END as user_segment
    FROM 'datalake/gold/user_daily_summary.parquet'
    ORDER BY total_spent DESC
    LIMIT 5;
"""

con = duckdb.connect(database=':memory:')
con.sql(query).show()


ANALYST QUERY (DuckDB Lakehouse)
┌─────────┬─────────────┬──────────────┐
│ user_id │ total_spent │ user_segment │
│  int64  │   double    │   varchar    │
├─────────┼─────────────┼──────────────┤
│     107 │      2082.0 │ VIP          │
│     103 │      1701.5 │ VIP          │
│     102 │      1688.0 │ VIP          │
│     109 │      1661.5 │ VIP          │
│     100 │      1609.0 │ VIP          │
└─────────┴─────────────┴──────────────┘



As we can see, this looks, feels and works like a normal datawarehouse or RDBMs just its sitting directly on top of our data files.

## QUESTIONS


1.   Why not just query the Bronze file directly?
2.   Why do we need a Gold layer? Why not just aggregate Silver every time?
3. If I find a bug in the Silver cleaning logic, what do I do?

